<a href="https://colab.research.google.com/github/PBMG14/Python_works/blob/main/WorkTill25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from abc import ABC, abstractmethod

class Boat(ABC):
    def __init__(self, boat_id, model, capacity, hourly_rate):
        self._boat_id = boat_id
        self._model = model
        self._capacity = capacity
        self._hourly_rate = hourly_rate
        self._is_available = True

    @abstractmethod
    def calculate_rental_cost(self, hours):
        pass

    def __str__(self):
        return f"Лодка: {self._model}, Вместимость: {self._capacity}"

    def __lt__(self, other):
        return self._hourly_rate < other._hourly_rate

    def __gt__(self, other):
        return self._hourly_rate > other._hourly_rate

    @property
    def boat_id(self):
        return self._boat_id

    @property
    def is_available(self):
        return self._is_available

    @is_available.setter
    def is_available(self, value):
        self._is_available = value

class Rowboat(Boat):
    def __init__(self, boat_id, model, capacity, hourly_rate, number_of_oars):
        super().__init__(boat_id, model, capacity, hourly_rate)
        self._number_of_oars = number_of_oars

    def calculate_rental_cost(self, hours):
        return self._hourly_rate * hours * (1 + 0.1 * self._number_of_oars)

    def __str__(self):
        return f"Весельная лодка: {self._model}, Весел: {self._number_of_oars}"

class Motorboat(Boat):
    def __init__(self, boat_id, model, capacity, hourly_rate, engine_power):
        super().__init__(boat_id, model, capacity, hourly_rate)
        self._engine_power = engine_power

    def calculate_rental_cost(self, hours):
        return self._hourly_rate * hours * (1 + 0.05 * self._engine_power)

    def __str__(self):
        return f"Моторная лодка: {self._model}, Мощность: {self._engine_power} л.с."

class Yacht(Boat):
    def __init__(self, boat_id, model, capacity, hourly_rate, luxury_level):
        super().__init__(boat_id, model, capacity, hourly_rate)
        self._luxury_level = luxury_level

    def calculate_rental_cost(self, hours):
        return self._hourly_rate * hours * (1 + 0.2 * self._luxury_level)

    def __str__(self):
        return f"Яхта: {self._model}, Уровень роскоши: {self._luxury_level}"

class Location:
    def __init__(self, city, address):
        self.city = city
        self.address = address

class Dock:
    def __init__(self, dock_id, location, capacity):
        self.dock_id = dock_id
        self.location = location
        self.capacity = capacity
        self.boats = []

    def add_boat(self, boat):
        if len(self.boats) < self.capacity:
            self.boats.append(boat)
            return True
        return False

    def remove_boat(self, boat_id):
        for boat in self.boats:
            if boat.boat_id == boat_id:
                self.boats.remove(boat)
                return True
        return False

    def get_available_boats(self):
        return [boat for boat in self.boats if boat.is_available]

class Rentable(ABC):
    @abstractmethod
    def rent_boat(self, boat_id, hours):
        pass

class Reportable(ABC):
    @abstractmethod
    def generate_report(self):
        pass

class LoggingMixin:
    def log(self, message):
        print(f"LOG: {message}")

class NotificationMixin:
    def notify(self, message):
        print(f"NOTIFY: {message}")

class BoatMeta(type):
    _registry = {}
    def __new__(cls, name, bases, namespace):
        new_class = super().__new__(cls, name, bases, namespace)
        if name != 'Boat':
            cls._registry[name.lower()] = new_class
        return new_class

class BoatFactory:
    @staticmethod
    def create_boat(boat_type, **kwargs):
        boat_class = BoatMeta._registry.get(boat_type.lower())
        if not boat_class:
            raise ValueError("Invalid boat type")
        return boat_class(**kwargs)

class Handler(ABC):
    def __init__(self, successor=None):
        self._successor = successor

    @abstractmethod
    def handle(self, request):
        pass

class DockOperator(Handler):
    def handle(self, request):
        if request['type'] == 'minor':
            print("Operator handled minor change")
            return True
        elif self._successor:
            return self._successor.handle(request)
        return False

class Manager(Handler):
    def handle(self, request):
        if request['type'] == 'cost':
            print("Manager handled cost change")
            return True
        elif self._successor:
            return self._successor.handle(request)
        return False

class Admin(Handler):
    def handle(self, request):
        print("Admin handled any change")
        return True

class RentalProcess(ABC):
    def rent(self, boat_id):
        self._check_availability(boat_id)
        self._create_contract(boat_id)
        return self._confirm()

    @abstractmethod
    def _check_availability(self, boat_id):
        pass

    @abstractmethod
    def _create_contract(self, boat_id):
        pass

    @abstractmethod
    def _confirm(self):
        pass

class OnlineRental(RentalProcess):
    def _check_availability(self, boat_id):
        print(f"Checking boat {boat_id} online")

    def _create_contract(self, boat_id):
        print(f"Creating online contract for {boat_id}")

    def _confirm(self):
        print("Confirmed online")
        return True

def check_permission(role):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if kwargs.get('user_role') != role:
                raise PermissionError("Access denied")
            return func(*args, **kwargs)
        return wrapper
    return decorator

class InvalidBoatError(Exception):
    pass

class PermissionDeniedError(Exception):
    pass

class Serializer:
    @staticmethod
    def to_dict(obj):
        return obj.__dict__

if __name__ == "__main__":
    rowboat = Rowboat("R1", "1", 2, 500, 2)
    motorboat = Motorboat("M1", "V2", 4, 1500, 50)

    location = Location("Город", "Набережная")
    dock = Dock("D1", location, 5)
    dock.add_boat(rowboat)
    dock.add_boat(motorboat)

    rowboat.is_available = False
    print(f"Доступные лодки: {[str(b) for b in dock.get_available_boats()]}")

    handler = DockOperator(Manager(Admin()))
    handler.handle({'type': 'minor'})
    handler.handle({'type': 'cost'})

    rental = OnlineRental()
    rental.rent("R1")

Доступные лодки: ['Моторная лодка: V2, Мощность: 50 л.с.']
Operator handled minor change
Manager handled cost change
Checking boat R1 online
Creating online contract for R1
Confirmed online
